# Universidad Nacional de Rosario - Facultad de Ciencias Exactas, Ingeniería y Agrimensura. 
## Tecnicatura Universitaria en Programación. 
### Procesamiento del Lenguaje Natural: Trabajo Práctico N°2.

---

**Autora: López Ceratto, Julieta : L-3311/1**

Se desarrolla a continuación el trabajo práctico final de la asignatura mencionada, cuyo objetivo es realizar un asistente destinado a brindar información o resolver consultas acerca del juego asignado.

# Parte 1: Base de datos de Grafos.

La base de datos de grafos se utiliza para modelar datos acerca del juego que no tienen que ver de manera estricta con su jugabilidad y que no son de amplia explicación, sino más bien relaciones entre distintas personas, otros juegos, premios y nacionalidades.
Incluye:
- Nombres alternativos.
- Diseñadores: nombres, nacionalidad, relaciones entre ellos, premios ganados, otros juegos hechos.
- Artistas: nombre, nacionalidad, relación con diseñadores, premios, otros juegos.
- Publicadores: nombres, nacionalidad.
- Juegos de su colección / familia: nombres, relación con diseñadores y artistas del mismo juego.

## Primeramente se debe extraer la información.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

### Encontrar links

In [2]:
## Url de partida
url_credits = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/credits"

In [3]:
# Como tienen carga con JavaScript utilizo selenuim.
# Configuración de Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en segundo plano
service = Service("src/chromedriver.exe")  # Especifica la ruta al chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)


In [37]:

# URL del sitio web
url = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/credits"
driver.get(url)



In [38]:
#Url 
def encontrar_url(dato:str)->list[url:str]:
    url = []
    finder = f'a.ng-binding[ng-href*="/boardgame{dato}/"]'
    links = driver.find_elements(By.CSS_SELECTOR, finder)
    # Extraer los href
    url = [link.get_attribute('href') for link in links]
    return url

url_designers = encontrar_url('designer')
url_artists = encontrar_url('artist')
url_publishers = encontrar_url('publisher')
url_related_to = encontrar_url('family')


In [39]:
print(url_designers,url_artists ,url_publishers ,url_related_to )

['https://boardgamegeek.com/boardgamedesigner/6940/inka-brand', 'https://boardgamegeek.com/boardgamedesigner/6941/markus-brand'] ['https://boardgamegeek.com/boardgameartist/12484/dennis-lohausen'] ['https://boardgamegeek.com/boardgamepublisher/3490/huch', 'https://boardgamegeek.com/boardgamepublisher/267/999-games', 'https://boardgamegeek.com/boardgamepublisher/2366/devir', 'https://boardgamegeek.com/boardgamepublisher/46544/dice-realm', 'https://boardgamegeek.com/boardgamepublisher/917/dv-games', 'https://boardgamegeek.com/boardgamepublisher/7496/egmont-polska', 'https://boardgamegeek.com/boardgamepublisher/37681/fabrika-igr', 'https://boardgamegeek.com/boardgamepublisher/12540/game-harbor', 'https://boardgamegeek.com/boardgamepublisher/17130/hot-games', 'https://boardgamegeek.com/boardgamepublisher/52507/nostalgia-iii', 'https://boardgamegeek.com/boardgamepublisher/105/r-and-r-games'] ['https://boardgamegeek.com/boardgamefamily/22783/admin-better-description-needed', 'https://boardga

### Encontrar informacion dentro de cada link

Defino información que quiero encontrar según cada link:
- Designer: Nombre, Pareja(si la tiene), Premios ganados, Otros juegos, nacionalidad.
- Artist: Nombre, Pareja(si la tiene), Premios ganados, Otros juegos, nacionalidad.
- Family: temática con las que se relaciona el juego.
- Publishers: Nombre.
- Nombres Alternativos: Nombre, idioma.
- Año de publicación.
- Premios: nombre de premio ganado, año.
            

In [64]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

In [ ]:
def encontrar_datos(entidad, lista_url)->dict:
    lista_dict = []
    if entidad == 'designer' or entidad == 'artist':
        
        for url in lista_url:
            driver.get(url)
            finder = f'a.ng-binding[ng-href*="/boardgame{entidad}/"]'
            links = driver.find_element(By.CSS_SELECTOR, finder)
            name_designer = links.text.strip()
            # Esperar hasta que la imagen esté cargada
            WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, "//img[contains(@alt, 'Microbadge:')]")))
            # Encontrar todas las etiquetas <img> con Microbadge
            img_elements = driver.find_elements(By.XPATH, "//img[contains(@alt, 'Microbadge:')]")
    
            # Verificar si hay más de un microbadge
            if len(img_elements) > 1:
                # Tomar el segundo microbadge (índice 1)
                second_img = img_elements[1]
                # Extraer el atributo 'alt'
                alt_text = second_img.get_attribute("alt")
                # Limpiar el texto para obtener solo el nombre
                partner = alt_text.replace("Microbadge: ", "").replace(" fan", "").strip()
            else:
                partner = ''
            # Buscar el enlace que contiene el nombre del premio
            try: 
                award = driver.find_element(By.CSS_SELECTOR, 'a.ng-binding[ng-href*="/boardgamehonor/"]')
                award = award.text.strip()  # Elimina espacios extra al inicio y fin
            except:
                award = ''
            
            
            see_all_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'See All')]"))
            )
            # Extraer el enlace del botón
            see_all_link = see_all_button.get_attribute("href")

             # Navegar al enlace extraído
            driver.get(see_all_link)

            # Esperar hasta que los nombres estén cargados
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//h3[@class='summary-item-title summary-item-title-separated fs-inherit mb-0']/a"))
            )

            # Extraer los primeros 10 nombres
            name_other_games = driver.find_elements(By.XPATH, "//h3[@class='summary-item-title summary-item-title-separated fs-inherit mb-0']/a")[:10]
            other_games = [name.text.strip() for name in name_other_games]
                    
            dict = {'Name': name_designer, 'Partner': partner, 'Award': award, 'Other Games': other_games}
            lista_dict.append(dict)
        
    elif entidad == 'publisher':

        for url in lista_url:
            driver.get(url)
            finder = f'a.ng-binding[ng-href*="/boardgame{entidad}/"]'
            links = driver.find_element(By.CSS_SELECTOR, finder)
            name_publisher = links.text.strip()
            dict = {'Name': name_publisher}
            lista_dict.append(dict)
            
    elif entidad == 'related_to':
        for url in lista_url:
            driver.get(url)
            finder = f'a.ng-binding[ng-href*="/boardgamefamily/"]'
            links = driver.find_element(By.CSS_SELECTOR, finder)
            subject = links.text.strip()
            # Dividir en temática y nombre
            if ":" in subject:
                partes = subject.split(":", 1)  # Dividir solo en el primer ":"
                tematica = partes[0].strip()  # Eliminar espacios en exceso
                nombre = partes[1].strip()  # Eliminar espacios en exceso
            else:
                tematica = subject.strip()
                nombre = ""  # Si no hay ":", dejar el nombre vacío
            
            # Crear el diccionario
            dict = {'Subject': tematica, 'Name': nombre}
            lista_dict.append(dict)
    elif entidad == 'alternative_names':
        driver.get(lista_url)
        try:
            # Esperar hasta que los nombres alternativos estén cargados
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//div[@ng-switch-when='alternatename']"))
            )
            
            # Encontrar todos los nombres alternativos
            alt_name_elements = driver.find_elements(By.XPATH, "//div[@ng-switch-when='alternatename']")
            alt_names = [element.text.strip() for element in alt_name_elements if element.text.strip()]
            
            for name in alt_names:
                dict = {'Alternative Name' : name}
                lista_dict.append(dict)
        except:
            lista_dict = None
        
            
        return lista_dict
    elif entidad == 'award_game':
        driver.get(lista_url)
        try:
            # URL inicial
            url = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges"
            driver.get(url)
            
            # Esperar a que los elementos <a> con premios estén cargados
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[@class='ng-binding']"))
            )
            
            # Encontrar todos los premios
            awards_elements = driver.find_elements(By.XPATH, "//a[@class='ng-binding'][contains(@ng-href, '/boardgamehonor/')]")
            
            # Lista para almacenar los premios procesados
            awards_list = []
            
            for award in awards_elements:
                text = award.text.strip()
                # Separar el año del nombre del premio usando una expresión regular
                match = re.match(r"(\d{4})\s*(.*)", text)
                if match:
                    year = match.group(1)
                    name = match.group(2).strip()
                    awards_list.append({'year': year, 'name': name})
            for award in awards_list:
                dict = {'Prize': award['name'], 'Year': award['year']}
                lista_dict.append(dict)
        except:
            lista_dict = None

    return lista_dict

In [72]:
dict_desingers =  encontrar_datos('designer', url_designers)
dict_artist = encontrar_datos('artist', url_artists)
dict_publishers = encontrar_datos('publisher', url_publishers)
dict_related_to = encontrar_datos('related_to', url_related_to)
dict_alternative_names = encontrar_datos('alternative_names', url)
dict_awards = encontrar_datos('award_game', url)

In [74]:
for designer in dict_desingers:
    designer['Nacionalidad'] = 'German'
for artist in dict_artist:
    artist['Nacionalidad'] = 'German'

In [75]:
dict_desingers

[{'Name': 'Inka Brand',
  'Partner': 'Markus Brand',
  'Award': '2012 Spiel des Jahres Kennerspiel des Jahres Winner',
  'Other Games': ['Andor Junior: Die Fährtenleserin Fennah / Der Fährtenleser Fenn (2020)',
   'Andor: The Family Fantasy Game (2020)',
   'Andor: The Family Fantasy Game – The Danger in the Shadows (2022)',
   'Bibi & Tina: Das Spiel zum Film (2013)',
   'Bitte nicht öffnen: Bissig! (2022)',
   'Black Stories: Das Spiel (2022)',
   'Blitzed! (2013)',
   'La Boca (2013)',
   'Bravo! (2019)',
   'Burgenland (2014)'],
  'Nacionalidad': 'German'},
 {'Name': 'Markus Brand',
  'Partner': 'Inka Brand',
  'Award': '2012 Spiel des Jahres Kennerspiel des Jahres Winner',
  'Other Games': ['Andor Junior: Die Fährtenleserin Fennah / Der Fährtenleser Fenn (2020)',
   'Andor: The Family Fantasy Game (2020)',
   'Andor: The Family Fantasy Game – The Danger in the Shadows (2022)',
   'Bibi & Tina: Das Spiel zum Film (2013)',
   'Bitte nicht öffnen: Bissig! (2022)',
   'Black Stories: 

In [76]:
dict_artist

[{'Name': 'Dennis Lohausen',
  'Partner': '',
  'Award': '',
  'Other Games': ['100! (2014)',
   '1001 Karawane (2007)',
   '1001 Karawane: Sonderchips (2007)',
   '112: Brandgefährlich (2017)',
   '15 (2011)',
   'The Academy (2023)',
   'Adventure Land (2023)',
   'Agent DOG (2024)',
   'Albion (2009)',
   'All Change! (and Chameleon) (2017)'],
  'Nacionalidad': 'German'}]

In [77]:
dict_publishers

[{'Name': 'HUCH!'},
 {'Name': '999 Games'},
 {'Name': 'Devir'},
 {'Name': 'Dice Realm'},
 {'Name': 'DV Games'},
 {'Name': 'Egmont Polska'},
 {'Name': 'Fabrika Igr'},
 {'Name': 'Game Harbor'},
 {'Name': 'HOT Games'},
 {'Name': 'nostalgia (III)'},
 {'Name': 'R&R Games'}]

In [78]:
dict_related_to

[{'Subject': 'Admin', 'Name': 'Better Description Needed!'},
 {'Subject': 'Country', 'Name': 'India'},
 {'Subject': 'Digital Implementations', 'Name': 'Tabletopia'},
 {'Subject': 'Digital Implementations', 'Name': 'Yucata'},
 {'Subject': 'Game', 'Name': 'Rajas of the Ganges'},
 {'Subject': 'Mechanism', 'Name': 'Dice Drafting'},
 {'Subject': 'Religious', 'Name': 'Hinduism'}]

In [79]:
dict_alternative_names

[{'Alternative Name': 'Ganges'},
 {'Alternative Name': 'I Ragià del Gange'},
 {'Alternative Name': "Raja's van de Ganges"},
 {'Alternative Name': 'Раджи Ганга'},
 {'Alternative Name': 'ガンジスの藩王'},
 {'Alternative Name': '갠지스의 라자'}]

In [80]:
dict_awards

[{'Prize': 'MinD-Spielepreis Complex Game Nominee', 'Year': '2019'},
 {'Prize': 'Juego del Año Recommended', 'Year': '2018'},
 {'Prize': 'International Gamers Award - General Strategy: Multi-player Winner',
  'Year': '2018'},
 {'Prize': 'International Gamers Award - General Strategy: Multi-player Nominee',
  'Year': '2018'},
 {'Prize': 'Meeples Choice Award Nominee', 'Year': '2017'}]

#### Detectar idioma de nombres alternativos

In [81]:
from langdetect import detect

In [84]:
for alt_name in dict_alternative_names:
    lang = detect(alt_name['Alternative Name'])
    alt_name['Language'] = lang

In [85]:
dict_alternative_names

[{'Alternative Name': 'Ganges', 'Language': 'de'},
 {'Alternative Name': 'I Ragià del Gange', 'Language': 'ca'},
 {'Alternative Name': "Raja's van de Ganges", 'Language': 'nl'},
 {'Alternative Name': 'Раджи Ганга', 'Language': 'bg'},
 {'Alternative Name': 'ガンジスの藩王', 'Language': 'ja'},
 {'Alternative Name': '갠지스의 라자', 'Language': 'ko'}]